### Calculate DG83 index
Method adapted from Pinheiro et al. (2019). This uses four thresholds to apply for the blocking criterea, which are applied in turn:
    
    AMPLITUDE - count the number of grid squares exceeding the var threshold (which may be -ve or +ve if one looks at the anomalies or not)
    AREA - for each blocked region count the number of grid cells at each latitude and so calculate the total area of each blocked region
    PERSISTENCE - measure how long the block persists for, and set a threshold for at least five days
    OVERLAP - count the number of days over which the contours for the blocked region overlap

If all of these criterea are met, then blocked_day = True. Else blocked_day = False.
    
These thresholds are applied to two datasets: 500hPa geopotentiel height anomalies, detrended wrt surface temperature (creating a measure similar to Dole and Gordon (1983)) and the seasonal anomaly of vertically averaged potential vorticity (similar to Schweirz (2004)).  The calculation of the anomaly fields and detrending has been done in a separate notebook, and uses ``cdo`` commands: https://code.mpimet.mpg.de/projects/cdo/embedded/cdo.pdf

In [1]:
#import iris
import netCDF4 as nc
import xarray
import numpy as np
import scipy
from collections import OrderedDict 
#from windspharm.iris import VectorWind
from scipy.integrate import simps
import glob
import matplotlib.pyplot as plt
#import PDF_funcs
import cartopy.crs as ccrs
from scipy import stats
from itertools import groupby
from operator import itemgetter
#for mapping the polygon on a sphere to a polygon on a flat surface to calculate area
import pyproj
import math
from shapely import geometry
import collections
from shapely.geometry import Point
from matplotlib.path import Path
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import xarray as xr

In [4]:
TM90 = xr.open_dataset("/rds/general/user/cmt3718/home/data/cmip6/TM90/blocked_lats/"
                "TM90_UKESM1-0-LL_r19i1p1f2_EURATL_ssp585_JJA_1850-2014_corrlon.nc")['blocking_index']

In [58]:
arr_list = sorted(glob.glob("/rds/general/project/carl_phd/live/carl/data/cmip6/*/ssp585/day/*/zg/*r180x91*"))[15:]

In [59]:
arr_list 
dir_zg_arr = []
for dir_list in arr_list:
    dir_zg = ((dir_list.split("_")[0]+"_"+dir_list.split("_")[1])[:-5])
    dir_zg_arr = np.append(dir_zg_arr, dir_zg)
    
unique_dir_arr=(np.unique(np.array(dir_zg_arr)))


In [81]:
unique_dir_arr[0].split("/")[9], unique_dir_arr[0].split("/")[12]

('CESM2-FV2', 'r1i1p1f1')

In [100]:
arr_list=glob.glob("/rds/general/project/carl_phd/live/carl/data/cmip6/*/ssp585/day/*/tas/*r180x91*")

for filedir in arr_list:
    if "_gn_" not in filedir:
        print(filedir)

/rds/general/project/carl_phd/live/carl/data/cmip6/AWI-CM-1-1-MR/ssp585/day/r1i1p1f1/tas/tas_day_AWI-CM-1-1-MR_ssp585_r1i1p1f1_18500101-19061231_r180x91.nc
/rds/general/project/carl_phd/live/carl/data/cmip6/ACCESS-ESM1-5/ssp585/day/r1i1p1f1/tas/tas_day_ACCESS-ESM1-5_ssp585_r1i1p1f1_18500101-21001231_r180x91.nc
/rds/general/project/carl_phd/live/carl/data/cmip6/GFDL-CM4/ssp585/day/r1i1p1f1/tas/tas_day_GFDL-CM4_historical_r1i1p1f1_gr1_18500101-18691231_r180x91.nc
/rds/general/project/carl_phd/live/carl/data/cmip6/BCC-CSM2-MR/ssp585/day/r1i1p1f1/tas/tas_day_BCC-CSM2-MR_ssp585_r1i1p1f1_18500101-21001231_r180x91.nc
/rds/general/project/carl_phd/live/carl/data/cmip6/BCC-CSM2-MR/ssp585/day/r1i1p1f1/tas/tas_day_BCC-CSM2-MR_ssp585_r1i1p1f1_19500101-21001231_r180x91.nc
/rds/general/project/carl_phd/live/carl/data/cmip6/CNRM-CM6-1/ssp585/day/r3i1p1f2/tas/tas_day_CNRM-CM6-1_ssp585_r3i1p1f2_gr_20150101-21001231_r180x91.nc
/rds/general/project/carl_phd/live/carl/data/cmip6/CNRM-CM6-1/ssp585/day/r4i1

In [164]:
arr_list=glob.glob("/rds/general/project/carl_phd/live/carl/data/cmip6/*/ssp585/day/*/zg/*")

dir_zg_arr = []
for dir_list in arr_list:
    if "BCC-CSM2-MR" not in arr_list:
        dir_zg = ((dir_list.split("_")[0]+"_"+dir_list.split("_")[1])[:-5])
        dir_zg_arr = np.append(dir_zg_arr, dir_zg)
    
unique_dir_arr=(np.unique(np.array(dir_zg_arr))) # use this directory to calculate LTDM
#unique_dir_arr
zg_files_list = []
for i in unique_dir_arr:
    file_arr=glob.glob(f"{i}*r180x91.nc*")
    for j in file_arr:
        if "_gn_" not in j:
            if "_gr" not in j:
                if "BCC-CSM2-MR" not in j:
                    zg_files_list = np.append(zg_files_list, j)

arr_list=glob.glob("/rds/general/project/carl_phd/live/carl/data/cmip6/*/ssp585/day/*/tas/*")

dir_zg_arr = []
for dir_list in arr_list:
    if "BCC-CSM2-MR" not in arr_list:
        dir_zg = ((dir_list.split("_")[0]+"_"+dir_list.split("_")[1])[:-5])
        dir_zg_arr = np.append(dir_zg_arr, dir_zg)
    
unique_dir_arr=(np.unique(np.array(dir_zg_arr))) # use this directory to calculate LTDM
#unique_dir_arr
tas_files_list = []
for i in unique_dir_arr:
    file_arr=glob.glob(f"{i}*r180x91.nc*")
    for j in file_arr:
        if "_gn_" not in j:
            if "_gr" not in j:
                if "BCC-CSM2-MR" not in j:
                    tas_files_list = np.append(tas_files_list, j)   
    
    

Load the files that are necessary  - zg_dtrnd_areaweighted_DG83_varbool is the important file, which is the end output of section1 and used in section 2 to apply the area threshold to.

In [40]:
#implelemnt the amplitude criterea (as above but normalised for when the file is loaded again)
#this is the file that is used from the rest of the notebook to implement the other blocking criterea
zg_dtrnd_areaweighted_DG83_varbool=xarray.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/"
                                            "500zg_JJA_era5_1979-2019_daymean_EurAR5_1x1_anom_dtrnd_wrt_tas_DG83_bool.nc")['500zg']

These are the variables used below; the definitions here are adapted from P19, but can be changed if desired.

In [7]:
persis_thresh = 5 # number of days over which blocking persists
JJA_days = 92
# overlap_val = minimum number of cells needed to say if there is overlap between two blocked regions
# set to 1 by default - any overlap between contours is acceptable
#may want to change depending on spatial and temporal resolution
overlap_val = 1

amp_thresh=1e-6 # following Schweirz et al 2004, who limited spatial resolution to 100 m
amp_thresh_anom=-1.2e-6 #threshold for a low VPV value for anomaly data

amp_min_anom_thresh = 100

area_thresh = 1e6 # following Pinheiro et al (2019)
JJA_days = 92
blocked_tiles_num_thresh = 30#24 lower threshold for area (implemented to make the area_test function marginally faster)

## 1. Alternative way of defining anomalies

Instead of calculating the anomaly for each day, calculate the anomaly wrt the long term daily mean. This can be derived by using the first five harmonics in the series to define the LDTM for 365 days in the year. This can then be used to subtract the data for each year. Following Pinheiro et al (2019) (and Grotjahn and Zhang (2014)), we have removed the leap years so that every year has 365 days. Note that this may cause issues with investigating blocking in DJF.

First calculate the mean gph leap year for each period

In [ ]:
#cdo del29feb 2mt_1979-2019_ERA5.nc 2mt_1979-2019_ERA5_nolp.nc

cdo timmean 2mt_1979-2019_ERA5_daymean.nc 2mt_1979-2019_ERA5_daymean_timmean.nc
cdo del29feb 2mt_1979-2019_ERA5_daymean.nc  2mt_1979-2019_ERA5_daymean_nolp.nc
cdo del29feb 2mt_1979-2019_ERA5_daymean_timmean.nc 2mt_1979-2019_ERA5_daymean_timmean_nolp.nc

In [3]:

mean_gph_nolp=xr.open_dataset("/rds/general/project/nowack_graven/live/carl/era5/2m_temperature/2mt_1979-2019_ERA5_daymean_nolp_ydayavg.nc")['t2m']
zg_file_nolp=xr.open_dataset("/rds/general/project/nowack_graven/live/carl/era5/2m_temperature/2mt_1979-2019_ERA5_daymean_nolp.nc")['t2m']

#xr.open_dataset("/rds/general/project/nowack_graven/live/carl/era5/2m_temperature/2mt_1979-2019_ERA5_daymean_nolp.nc")['t2m']

#mean_gph_nolp=xr.open_dataset("/rds/general/project/nowack_graven/live/carl/era5/2m_temperature/2mt_1979-2019_ERA5_daymean_timmean_nolp.nc")['t2m']


In [8]:
import xarray as xr
mean_gph_nolp=xr.open_dataset("/rds/general/project/carl_phd/live/carl/data/cmip6/BCC-CSM2-MR/ssp585/day/r1i1p1f1/zg/"
"500zg_day_BCC-CSM2-MR_ssp585_r1i1p1f1_19500101-21001231_r180x91.nc")['zg'].squeeze()

In [9]:
mean_gph_nolp

<xarray.DataArray 'zg' (time: 55115, lat: 91, lon: 180)>
[902783700 values with dtype=float32]
Coordinates:
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 84.0 86.0 88.0 90.0
    plev     float64 5e+04
  * time     (time) object 1950-01-01 12:00:00 ... 2100-12-31 12:00:00
Attributes:
    standard_name:  geopotential_height
    long_name:      Geopotential Height
    units:          m
    comment:        Geopotential is the sum of the specific gravitational pot...
    original_name:  Z3
    cell_methods:   time: mean (interval: 5 minutes)
    cell_measures:  area: areacella

In [9]:
LTDM_arr = np.zeros(mean_gph_nolp.shape)
LTDM_arr.shape

(55115, 91, 180)

In [17]:
mean_gph_nolp.shape

(365, 91, 180)

In [18]:
mean_gph_nolp=xr.open_dataset("/rds/general/project/carl_phd/live/carl/data/cmip6/BCC-CSM2-MR/ssp585/day/r1i1p1f1/zg/"
"500zg_day_BCC-CSM2-MR_ssp585_r1i1p1f1_19500101-21001231_r180x91_ydayavg.nc")['zg'].squeeze()

def calc_LTDM(test_lat, test_lon):
    #calcualte the discrete Fourier transform for each of the 365 days
    y = mean_gph_nolp[:,test_lat, test_lon]
    rft = np.fft.fft(y)
    #only the first 6 harmonics are selected
    rft[6:] = 0   # set the subsequent harmonics to zero
    LTDM = np.fft.ifft(rft)
    return np.real(LTDM)

LTDM_arr = np.zeros(mean_gph_nolp.shape)
for test_lat in range(mean_gph_nolp.shape[1]):
    for test_lon in range(mean_gph_nolp.shape[2]):
        LTDM_arr[:,test_lat,test_lon]=calc_LTDM(test_lat, test_lon)
#LTDM_arr_xr = xarray.DataArray(LTDM_arr, coords=(mean_gph_nolp['time'][:365],mean_gph_nolp['lat'],mean_gph_nolp['lon']))

In [20]:
LTDM_arr_xr = xarray.DataArray(LTDM_arr, coords=(mean_gph_nolp['time'][:365],mean_gph_nolp['lat'],mean_gph_nolp['lon']))
LTDM_arr_xr.to_netcdf("/rds/general/project/carl_phd/live/carl/data/cmip6/BCC-CSM2-MR/ssp585/day/r1i1p1f1/zg/"
                      "500zg_day_BCC-CSM2-MR_ssp585_r1i1p1f1_19500101-21001231_r180x91_ydayavg.nc")


In [29]:

LTDM_arr_xr = xarray.DataArray(LTDM_arr, coords=(mean_gph_nolp['time'][:365],mean_gph_nolp['lat'],mean_gph_nolp['lon']))
LTDM_arr_xr.to_netcdf("/rds/general/project/nowack_graven/live/carl/era5/2m_temperature/2mt_1x1_1979-2019_daymean_LTDM_nolp.nc")

In [23]:
LTDM_arr_xr.to_netcdf(f"/rds/general/project/carl_phd/live/carl/data/cmip6/BCC-CSM2-MR/ssp585/day/r1i1p1f1/zg/"
                      "500zg_day_BCC-CSM2-MR_ssp585_r1i1p1f1_19500101-21001231_r180x91_LTDM.nc")
#zg_file_no_lp_anom = [(zg_file_no_lp[i*365:(i+1)*365,:,:]-LTDM_arr_xr.values) for i in range(41)] 
#LTDM_arr_tot = np.concatenate(LTDM_arr, axis = 0)
#zg_file_no_lp_anom_xr = xarray.concat(zg_file_no_lp_anom, 'time')
#zg_file_no_lp_anom_xr.to_netcdf("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/tot/500zg_1x1_1979-2019_daymean_LTDManom_nolp.nc")

In [33]:
#LTDM_arr_xr.to_netcdf(f"/rds/general/project/carl_phd/live/carl/data/era5/day/zg/tot/500zg_1979-2019_daymean_LTDM.nc")
zg_file_nolp_anom = [(zg_file_nolp[i*365:(i+1)*365,:,:]-LTDM_arr_xr.values) for i in range(41)] 
#LTDM_arr_tot = np.concatenate(LTDM_arr, axis = 0)
#zg_file_no_lp_anom_xr = xarray.concat(zg_file_no_lp_anom, 'time')
#zg_file_no_lp_anom_xr.to_netcdf("/rds/general/project/nowack_graven/live/carl/era5/2m_temperature/2mt_1x1_1979-2019_daymean_LTDM_nolp_anom.nc")

In [35]:
zg_file_nolp_anom_xr = xarray.concat(zg_file_nolp_anom, 'time')
zg_file_nolp_anom_xr.to_netcdf("/rds/general/project/nowack_graven/live/carl/era5/2m_temperature/2mt_1x1_1979-2019_daymean_LTDM_nolp_anom.nc")

/home/cmt3718/anaconda3/envs/odin/lib/python3.6/site-packages/xarray/coding/times.py:255: FutureWarning: the 'box' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'box'
  unique_timedeltas = pd.to_timedelta(unique_timedeltas, box=False)


In [22]:
LTDM_arr_xr = xarray.DataArray(LTDM_arr, coords=(mean_gph_nolp['time'][:365],mean_gph_nolp['latitude'],mean_gph_nolp['longitude']))

KeyError: 'latitude'

In [4]:
#the .groupby('time.dayofyear') groups by the ordinal day
#which is an integer that specifies the number of days since 31st December the previous year
#as such, the value of it differs for leap years, so to calculate the group mean
#a new modified variable needs to be created
#this has been adapted from https://github.com/pydata/xarray/issues/1844
zg_file=xarray.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/tot/500zg_1x1_1979-2019_daymean.nc")['z']/9.80665
dates = zg_file['time']

da = zg_file['time']

not_leap_year = xarray.DataArray(~da.indexes['time'].is_leap_year, coords=da.coords)

march_or_later = da.time.dt.month >= 3

ordinal_day = da.time.dt.dayofyear

modified_ordinal_day = ordinal_day + (not_leap_year & march_or_later)

modified_ordinal_day = modified_ordinal_day.rename('modified_ordinal_day')

mean_gph = zg_file.groupby(modified_ordinal_day).mean('time')
#to remove the leap year
mean_gph_nolp = xarray.concat([mean_gph[:59,:,:], mean_gph[60:,:,:]], 'modified_ordinal_day')

Derive the long term daily mean for each value in the dataset

# 2. Calculate and save the amplitude critereon

This produces the zg_dtrnd_areaweighted_DG83_varbool file which is loaded above.

Note the ``calc_DG83_area_weighted_bool.py`` and ``detrend_zg_wrt_tas.py`` scripts are also used to apply the transformation of eq. (3) from P19 and to linearly detrend the data with respect to the surface temperature, to remove the local mean.

the anomaly file is calculated using the command: 
``cdo -ydaysub infile -ydayavg infile outfile``
cdo vertmean fldmean and timstd are used to calculate the vertical mean, field (spatial) mean and the standard deviation over time.

Following P19, there is a varying anomaly threshold (equation 5) that is applied to both datasets, which is then used to calculate whether or not each grid cell exceeds its local theshold.

Note: this doesn't need to be run since these files have been saved below.


In [3]:
LTDM_anom_nolp=xr.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/500zg_1x1_1979-2019_daymean_LTDManom_nolp.nc")['z']


xr.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/500zg_JJA_1x1_1979-2019_daymean_LTDManom_EurAR5plus5.nc")['z']


<xarray.DataArray 'z' (time: 3772, latitude: 56, longitude: 61)>
[12885152 values with dtype=float64]
Coordinates:
  * longitude  (longitude) float32 -15.0 -14.0 -13.0 -12.0 ... 43.0 44.0 45.0
  * latitude   (latitude) float32 80.0 79.0 78.0 77.0 ... 28.0 27.0 26.0 25.0
  * time       (time) datetime64[ns] 1979-06-01T10:30:00 ... 2019-08-31T10:30:00

In [8]:
xr.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/500zg_JJA_1x1_1979-2019_daymean_LTDManom_EurAR5.nc")['z']

<xarray.DataArray 'z' (time: 3772, latitude: 46, longitude: 51)>
[8849112 values with dtype=float64]
Coordinates:
  * longitude  (longitude) float32 -10.0 -9.0 -8.0 -7.0 ... 37.0 38.0 39.0 40.0
  * latitude   (latitude) float32 75.0 74.0 73.0 72.0 ... 33.0 32.0 31.0 30.0
  * time       (time) datetime64[ns] 1979-06-01T10:30:00 ... 2019-08-31T10:30:00

In [ ]:
"/rds/general/project/nowack_graven/live/carl/era5/2m_temperature/*glo.nc*"

In [10]:
xr.open_dataset("/rds/general/project/nowack_graven/live/carl/era5/2m_temperature/ERA5_1979_2m_temperature_glo.nc")['t2m']

<xarray.DataArray 't2m' (time: 2920, latitude: 181, longitude: 360)>
[190267200 values with dtype=float32]
Coordinates:
  * longitude  (longitude) float32 0.0 1.0 2.0 3.0 ... 356.0 357.0 358.0 359.0
  * latitude   (latitude) float32 90.0 89.0 88.0 87.0 ... -88.0 -89.0 -90.0
  * time       (time) datetime64[ns] 1979-01-01 ... 1979-12-31T21:00:00
Attributes:
    units:      K
    long_name:  2 metre temperature

In [6]:
xr.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/2t/2mt_1979-2019_1x1_EurAR5_daymean_LTDM.nc")

<xarray.Dataset>
Dimensions:                        (latitude: 46, longitude: 51, time: 365)
Coordinates:
  * time                           (time) datetime64[ns] 1979-01-01T10:30:00 ... 1979-12-31T10:30:00
  * latitude                       (latitude) float32 75.0 74.0 ... 31.0 30.0
  * longitude                      (longitude) float32 -10.0 -9.0 ... 39.0 40.0
Data variables:
    __xarray_dataarray_variable__  (time, latitude, longitude) float64 ...

In [3]:
xr.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/500zg_JJA_1x1_1979-2019_daymean_LTDManom.nc")

<xarray.Dataset>
Dimensions:    (latitude: 181, longitude: 360, time: 3772)
Coordinates:
  * longitude  (longitude) float32 0.0 1.0 2.0 3.0 ... 356.0 357.0 358.0 359.0
  * latitude   (latitude) float32 90.0 89.0 88.0 87.0 ... -88.0 -89.0 -90.0
  * time       (time) datetime64[ns] 1979-06-01T10:30:00 ... 2019-08-31T10:30:00
Data variables:
    z          (time, latitude, longitude) float64 ...
Attributes:
    CDI:          Climate Data Interface version 1.9.0 (http://mpimet.mpg.de/...
    Conventions:  CF-1.6
    history:      Tue Jan 28 09:55:50 2020: cdo select,season=JJA /rds/genera...
    CDO:          Climate Data Operators version 1.9.0 (http://mpimet.mpg.de/...

In [ ]:
cdo sellonlatbox,-15,45,25,80 2mt_1x1_1979-2019_daymean_LTDM_nolp_anom.nc 2mt_1x1_1979-2019_daymean_LTDM_nolp_anom_Eurplus5.nc
cdo fldmean 2mt_1x1_1979-2019_daymean_LTDM_nolp_anom_Eurplus5.nc 2mt_1x1_1979-2019_daymean_LTDM_nolp_anom_Eurplus5_fldmean.nc







In [37]:
xarray.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/2t/"
                    "2mt_1x1_1979-2019_daymean_LTDM_nolp_anom_Eurplus5.nc")

<xarray.Dataset>
Dimensions:    (latitude: 56, longitude: 61, time: 14965)
Coordinates:
  * longitude  (longitude) float32 -15.0 -14.0 -13.0 -12.0 ... 43.0 44.0 45.0
  * latitude   (latitude) float32 80.0 79.0 78.0 77.0 ... 28.0 27.0 26.0 25.0
  * time       (time) object 1979-01-01 10:30:00 ... 2019-12-31 10:30:00
Data variables:
    t2m        (time, latitude, longitude) float64 ...
Attributes:
    CDI:          Climate Data Interface version 1.9.0 (http://mpimet.mpg.de/...
    Conventions:  CF-1.6
    history:      Thu Mar 05 12:16:48 2020: cdo sellonlatbox,-15,45,25,80 2mt...
    CDO:          Climate Data Operators version 1.9.0 (http://mpimet.mpg.de/...

In [55]:
#load zg and tas files
zg_file=xarray.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/500zg_JJA_era5_1979-2019_3hr_1x1_daymean_anom_EurAR5.nc")['z']
tas_file=xarray.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/2t/2t_JJA_era5_1979-2019_3hr_1x1_daymean_anom_EurAR5.nc")['t2m']
#for the LTDM anom method
#zg_anom=xarray.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/tot/500zg_JJA_1x1_1979-2019_daymean_LTDManom_EurAR5.nc")['z']
#tas_file=xarray.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/2t/2t_JJA_era5_1979-2019_3hr_1x1_daymean_anom_EurAR5.nc")['t2m']

#for the LTDM anom method
zg_anom=xarray.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/500zg_JJA_1x1_1979-2019_daymean_LTDManom_Eurplus5.nc")['z']
tas_file=xarray.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/2t/2mt_1x1_1979-2019_daymean_LTDM_nolp_anom_Eurplus5.nc")['t2m']

zg_anom_fldmean=xarray.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/"
                                    "500zg_JJA_1x1_1979-2019_daymean_LTDManom_Eurplus5_fldmean.nc")['z'].squeeze()
tas_anom_fldmean=xarray.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/2t/"
                                     "2mt_JJA_1x1_1979-2019_daymean_LTDM_nolp_anom_Eurplus5_fldmean.nc")['t2m'].squeeze()


#500zg_JJA_1x1_1979-2019_daymean_LTDManom_EurAR5_fldmean.nc

#cdo -b 32 sellonlatbox,-10,40,30,75 -ydaysub 2t_JJA_era5_1979-2019_3hr_1x1_daymean.nc
#-ydayavg 2t_JJA_era5_1979-2019_3hr_1x1_daymean.nc 2t_JJA_era5_1979-2019_3hr_1x1_daymean_anom_EurAR5.nc


#load weighted anomaly means to calculate the tas/zg trend
#tas_anom_fldmean = xarray.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/2t/2t_JJA_era5_1979-2019_3hr_1x1_daymean_EurAR5_fldmean_anom.nc")['t2m'][:,0,0]
#zg_anom_fldmean = xarray.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/tot/500zg_JJA_1x1_1979-2019_daymean_LTDManom_EurAR5_fldmean.nc")['z'][:,0,0]

In [56]:
tas_anom_fldmean, zg_anom_fldmean

(<xarray.DataArray 't2m' (time: 3772)>
 array([3.146512, 3.192198, 3.018844, ..., 5.440489, 5.239959, 5.086525])
 Coordinates:
     lon      float32 0.0
     lat      float32 0.0
   * time     (time) object 1979-06-01 10:30:00 ... 2019-08-31 10:30:00,
 <xarray.DataArray 'z' (time: 3772)>
 array([ 92.872425, 107.02846 , 101.841373, ..., 116.81977 , 113.61184 ,
         92.320637])
 Coordinates:
     lon      float32 0.0
     lat      float32 0.0
   * time     (time) datetime64[ns] 1979-06-01T10:30:00 ... 2019-08-31T10:30:00)

In [39]:
time, lat, lon = zg_anom['time'], zg_anom['latitude'], zg_anom['longitude']

In [40]:
#function to detrend the data with respect to the global mean surface temperature
def dtrnd_wrt_tas(anom_var, tas_file, var = "zg"):
    if var == "psl":
        lat_len = anom_var.shape[1]
        lon_len = anom_var.shape[2]
    if var == "zg":    
        lat_len = anom_var.shape[2]
        lon_len = anom_var.shape[3]        
    dtrnd_tas_arr = np.zeros((anom_var.shape[0],lat_len,lon_len))
    x = np.array(tas_file.variables['tas'][cutoff:,0,0]) - 273.15
    for i in range(lat_len):
        for j in range(lon_len):
            if var == "zg":
                y = zg_file['zg'][cutoff:,plev,i,j]
            if var == "psl":
                y = psl_file['psl'][cutoff:,i,j]/100 
            coeffs_glo_mean_tas = np.polyfit(x, y, 1)
            m_arr = np.arange(len(y))
            dtrnd_tas_arr[:,i,j] = y - coeffs_glo_mean_tas[0]*x - coeffs_glo_mean_tas[1]
    return dtrnd_tas_arr

In [52]:
x, y

(<xarray.DataArray 't2m' (time: 14965)>
 array([-6.90991 , -7.247134, -7.578977, ..., -2.585275, -2.84168 , -3.125893])
 Coordinates:
     lon      float32 0.0
     lat      float32 0.0
   * time     (time) object 1979-01-01 10:30:00 ... 2019-12-31 10:30:00,
 <xarray.DataArray 'z' (time: 3772)>
 array([ 92.872425, 107.02846 , 101.841373, ..., 116.81977 , 113.61184 ,
         92.320637])
 Coordinates:
     lon      float32 0.0
     lat      float32 0.0
   * time     (time) datetime64[ns] 1979-06-01T10:30:00 ... 2019-08-31T10:30:00)

In [57]:
#calculate trend
x=tas_anom_fldmean
y=zg_anom_fldmean
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
#detrend data - takes a few seconds
zg_dtrnd = [zg - tas_anom_fldmean[i]*slope - intercept for i, zg in enumerate((zg_anom[:]))]

In [58]:
#concatenate the set of xarrays created above - also takes a few seconds
zg_dtrnd_concat = xarray.concat(zg_dtrnd, "time")

In [6]:
zg_anom_dtrnd_xr = xarray.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/"
                                       "500zg_JJA_era5_1979-2019_daymean_EurAR5_1x1_anom_dtrnd_wrt_tas.nc")

In [62]:
zg_dtrnd_concat

<xarray.DataArray (time: 3772, latitude: 56, longitude: 61)>
array([[[-87.500906, -83.967214, ...,  21.394114,  21.067454],
        [-76.414968, -72.228445, ...,  28.33845 ,  27.50593 ],
        ...,
        [-60.463796, -56.375417, ..., -25.16448 , -23.455007],
        [-56.993093, -53.502371, ..., -24.078054, -23.029226]],

       [[-66.068383, -62.895531, ...,  61.955543,  61.050758],
        [-68.652856, -65.463402, ...,  77.568824,  76.215797],
        ...,
        [-48.255883, -40.976586, ..., -34.26809 , -29.914086],
        [-41.429223, -35.905297, ..., -33.40725 , -29.709496]],

       ...,

       [[ 96.843815,  96.997135, ..., 197.304264, 197.163639],
        [ 77.83112 ,  77.758366, ..., 208.259342, 208.703678],
        ...,
        [-25.06097 , -24.711361, ..., -44.941341, -46.220638],
        [-30.025326, -28.977474, ..., -53.822201, -55.048763]],

       [[152.214991, 150.538722, ..., 182.146632, 183.220362],
        [126.363917, 124.076808, ..., 184.911769, 186.551417],

In [60]:
zg_dtrnd_concat.to_netcdf("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/500zg_JJA_era5_1979-2019_daymean_Eurplus5_1x1_LTDManom_dtrnd_wrt_tas.nc")

In [63]:
#define the attrs for the new xarray
zg_attrs = OrderedDict()
zg_attrs['units'] = 'm'
zg_attrs['long_name'] = 'Geopotential height'
zg_attrs['level'] = '500 hPa'
zg_attrs['dataset'] = 'era5'
zg_attrs['detrended'] = '500 hPa vs surface temperature Europe fldmeam anom'
zg_attrs['date_range'] = 'JJA 1979-2019'
#transfer the concatenated xarray into a DataArray to redefine the axes and save
zg_anom_dtrnd_xr = xarray.DataArray(zg_dtrnd_concat, coords=(zg_anom['time'],zg_anom['latitude'],zg_anom['longitude']), name = "500zg", attrs=zg_attrs)
#save here because this file is also useful for the SOM plotting
zg_anom_dtrnd_xr.to_netcdf("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/"
                           "500zg_JJA_era5_1979-2019_daymean_Eurplus5_1x1_LTDManom_dtrnd_wrt_tas.nc")

In [64]:
zg_anom_dtrnd_xr

<xarray.DataArray '500zg' (time: 3772, latitude: 56, longitude: 61)>
array([[[-87.500906, -83.967214, ...,  21.394114,  21.067454],
        [-76.414968, -72.228445, ...,  28.33845 ,  27.50593 ],
        ...,
        [-60.463796, -56.375417, ..., -25.16448 , -23.455007],
        [-56.993093, -53.502371, ..., -24.078054, -23.029226]],

       [[-66.068383, -62.895531, ...,  61.955543,  61.050758],
        [-68.652856, -65.463402, ...,  77.568824,  76.215797],
        ...,
        [-48.255883, -40.976586, ..., -34.26809 , -29.914086],
        [-41.429223, -35.905297, ..., -33.40725 , -29.709496]],

       ...,

       [[ 96.843815,  96.997135, ..., 197.304264, 197.163639],
        [ 77.83112 ,  77.758366, ..., 208.259342, 208.703678],
        ...,
        [-25.06097 , -24.711361, ..., -44.941341, -46.220638],
        [-30.025326, -28.977474, ..., -53.822201, -55.048763]],

       [[152.214991, 150.538722, ..., 182.146632, 183.220362],
        [126.363917, 124.076808, ..., 184.911769, 186.

In [65]:
#code for applying the latitudinal coefficient
#as per Dole and Gordon (1983), need to normalize the detrended 500hPa geopotential height data by a latitutdinal coefficient (eq1 from DG 1983)
lat_coeff = np.sin(np.radians(45))
#use xarrays in list comprehension and then concatenante thm later to form the list
zg_dtrnd_norm = [lat_coeff*(zg_anom_dtrnd_xr[:,i,:])/np.sin(np.radians(lat)) for i, lat in enumerate(np.array(lat))]

#define the attrs for the new xarray
zg_attrs = OrderedDict()
zg_attrs['units'] = 'm'
zg_attrs['long_name'] = 'Geopotential height'
zg_attrs['level'] = '500 hPa'
zg_attrs['dataset'] = 'era5'
zg_attrs['detrended'] = '500 hPa vs surface temperature EurAR5 fldmean anom'
zg_attrs['norm'] = 'latitudinal coefficient (eq1 from Dole and Gordon 1983)'
zg_attrs['date_range'] = 'JJA 1979-2018'

zg_dtrnd_norm_concat = xarray.concat(zg_dtrnd_norm, "latitude").transpose('time', 'latitude', 'longitude')
#save here to then use cdo timstd to calculate the time standard deviation to create the array of normalized anomalies
#(see 2.2.4 in Pinheiro et al. (2019))
zg_dtrnd_norm_concat.to_netcdf("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/"
                           "500zg_JJA_era5_1979-2019_daymean_Eurplus5_1x1_LTDManom_dtrnd_wrt_tas_DG83.nc")



#("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/tot/"
#                               "500zg_JJA_era5_1979-2019_daymean_EurAR5_1x1_LTDManom_dtrnd_wrt_tas_DG83.nc")


Then run ``cdo timstd /rds/general/project/carl_phd/live/carl/data/era5/day/zg/500zg_JJA_era5_1979-2019_daymean_EurAR5_1x1_anom_dtrnd_wrt_tas_DG83.nc /rds/general/project/carl_phd/live/carl/data/era5/day/zg/500zg_JJA_era5_1979-2019_daymean_EurAR5_1x1_anom_dtrnd_wrt_tas_DG83_timstd.nc`` on the dataset to calculate the time std efficiently (it may be worth using the subprocess library to integrate the bash script commands with this ipynb) to implement the normalized anomaly

In [82]:
zg_dtrnd_norm_concat

<xarray.DataArray '500zg' (time: 3772, latitude: 46, longitude: 51)>
array([[[ -19.024953,  -14.202639, ...,   15.995887,   12.627666],
        [  -9.046545,   -3.793881, ...,   20.378644,   16.193107],
        ...,
        [-121.58304 , -117.623154, ..., -113.253671, -109.402385],
        [-124.210894, -121.396968, ..., -116.11093 , -113.487592]],

       [[ -74.006541,  -72.279716, ...,   67.6553  ,   67.604186],
        [ -69.200245,  -67.249532, ...,   74.467147,   74.286479],
        ...,
        [-104.427953,  -90.650472, ..., -146.018487, -142.672662],
        [-102.187412,  -88.509315, ..., -147.897308, -145.389288]],

       ...,

       [[ -52.060881,  -51.875366, ...,  141.285607,  143.290525],
        [ -73.9818  ,  -73.87225 , ...,  142.477941,  144.630154],
        ...,
        [ -45.666772,  -42.143968, ...,  -29.268138,  -27.856335],
        [ -44.200086,  -42.988199, ...,  -38.585354,  -36.758202]],

       [[ -25.381098,  -25.432213, ...,  101.905311,  106.16822 ],
  

In [110]:
timstd_arr = np.zeros((46,51))

for i in range(46):
    for j in range(51):
        timstd_arr[i,j] = np.std(zg_dtrnd_norm_concat[:,i,j])


In [111]:
timstd_arr

array([[66.4400665 , 66.70864304, 66.99215774, ..., 81.25366635,
        81.54224194, 81.82385509],
       [66.53622379, 66.80499571, 67.08299731, ..., 80.12357165,
        80.42045015, 80.71467449],
       [66.9259836 , 67.20141844, 67.48638179, ..., 79.02749678,
        79.33280514, 79.64011473],
       ...,
       [55.37656294, 55.06929105, 54.51172693, ..., 44.25033883,
        43.69257469, 43.07710164],
       [53.68557145, 53.56784308, 53.1719918 , ..., 42.71974476,
        42.1468252 , 41.50294956],
       [51.78974339, 51.45974967, 50.75288479, ..., 41.40845938,
        40.71139149, 40.11832077]])

In [112]:
#can use cdo timstd or manually calculate the tims std since there seem to be regridding issues
zg_dtrnd_norm_concat_timstd = [np.std(zg_dtrnd_norm_concat[:,i,j]) for i in range(46) for j in range(51)]
zg_dtrnd_norm_concat_timstd_ds = (xr.concat(zg_dtrnd_norm_concat_timstd, "value")).to_dataset()
#convert to numpy array to apply the reshape operation
norm_anom = zg_dtrnd_norm_concat_timstd_ds['500zg'].values.reshape(46,51)

In [130]:
xarray.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/"
                    "500zg_JJA_era5_1979-2019_daymean_EurAR5_1x1_anom_dtrnd_wrt_tas_DG83_timstd.nc")['500zg']

<xarray.DataArray '500zg' (time: 1, latitude: 46, longitude: 51)>
array([[[638.784487, 641.273553, ..., 770.289618, 772.654946],
        [639.748873, 642.272347, ..., 758.300993, 760.731455],
        ...,
        [489.400102, 486.628736, ..., 399.542193, 393.707649],
        [470.489308, 465.991472, ..., 388.13654 , 382.714808]]])
Coordinates:
  * longitude  (longitude) float32 -10.0 -9.0 -8.0 -7.0 ... 37.0 38.0 39.0 40.0
  * latitude   (latitude) float32 75.0 74.0 73.0 72.0 ... 33.0 32.0 31.0 30.0
  * time       (time) datetime64[ns] 1999-07-16T22:30:00

In [65]:
xarray.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/tot/500zg_JJA_era5_1979-2019_daymean_EurAR5_1x1_LTDManom_dtrnd_wrt_tas_DG83_timstd.nc")

<xarray.Dataset>
Dimensions:    (bnds: 2, latitude: 46, longitude: 360, time: 1)
Coordinates:
  * longitude  (longitude) float32 0.0 1.0 2.0 3.0 ... 356.0 357.0 358.0 359.0
  * latitude   (latitude) float32 90.0 89.0 88.0 87.0 ... 48.0 47.0 46.0 45.0
  * time       (time) datetime64[ns] 1999-07-16T22:30:00
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    500zg      (time, latitude, longitude) float64 ...
Attributes:
    CDI:          Climate Data Interface version 1.9.0 (http://mpimet.mpg.de/...
    Conventions:  CF-1.6
    history:      Tue Jan 28 10:18:28 2020: cdo timstd 500zg_JJA_era5_1979-20...
    CDO:          Climate Data Operators version 1.9.0 (http://mpimet.mpg.de/...

In [78]:
norm_anom_orig = xarray.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/"
                                "500zg_JJA_era5_1979-2019_daymean_EurAR5_1x1_anom_dtrnd_wrt_tas_DG83_timstd.nc")

In [132]:
norm_anom*9.80616

array([[651.52192254, 654.15562704, 656.93581756, ..., 796.78645283,
        799.61627122, 802.37781479],
       [652.46485626, 655.10047669, 657.82660491, ..., 785.70456339,
        788.61580142, 791.50101238],
       [656.28690331, 658.98786146, 661.78225766, ..., 774.95627785,
        777.95018043, 780.96370746],
       ...,
       [543.03143639, 540.01827911, 534.55071611, ..., 433.92590259,
        428.45637824, 422.42095104],
       [526.44930333, 525.29484012, 521.41305911, ..., 418.9166523 ,
        413.29851141, 406.98456384],
       [507.85851001, 504.62253885, 497.69090873, ..., 406.05797803,
        399.22241873, 393.40667243]])

In [123]:
norm_anom_xr

<xarray.DataArray (latitude: 46, longitude: 51)>
array([[66.440067, 66.708643, 66.992158, ..., 81.253666, 81.542242, 81.823855],
       [66.536224, 66.804996, 67.082997, ..., 80.123572, 80.42045 , 80.714674],
       [66.925984, 67.201418, 67.486382, ..., 79.027497, 79.332805, 79.640115],
       ...,
       [55.376563, 55.069291, 54.511727, ..., 44.250339, 43.692575, 43.077102],
       [53.685571, 53.567843, 53.171992, ..., 42.719745, 42.146825, 41.50295 ],
       [51.789743, 51.45975 , 50.752885, ..., 41.408459, 40.711391, 40.118321]])
Coordinates:
  * latitude   (latitude) float32 75.0 74.0 73.0 72.0 ... 33.0 32.0 31.0 30.0
  * longitude  (longitude) float32 -10.0 -9.0 -8.0 -7.0 ... 37.0 38.0 39.0 40.0

In [118]:
norm_anom_xr = xr.DataArray(norm_anom, coords=(zg_dtrnd_norm_concat['latitude'], zg_dtrnd_norm_concat['longitude']))

In [121]:
norm_anom_xr.to_netcdf("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/tot/"
    "500zg_JJA_era5_1979-2019_daymean_EurAR5_1x1_LTDManom_dtrnd_wrt_tas_DG83_timstd.nc")

In [66]:
#load the file for the normalised anomalies
#norm_anom = xarray.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/"
#                                "500zg_JJA_era5_1979-2019_daymean_EurAR5_1x1_anom_dtrnd_wrt_tas_DG83_timstd.nc")
norm_anom = xarray.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/tot/"
                                "500zg_JJA_era5_1979-2019_daymean_EurAR5_1x1_LTDManom_dtrnd_wrt_tas_DG83_timstd.nc")


In [133]:
#apply the varying anomaly threshold from Pinheiro et al. 2019
#set values below minimum equal to minimum, as defined in P19
#note that the signf here are opposite - we are concerned with very low PV anomalies instead of
#high geopotential height anomalies
amp_min_anom_thresh = 100
var_anom_thresh_arr = norm_anom_xr[:,:]*1.5*9.80616
var_anom_minthresh_arr = np.asarray(np.where(var_anom_thresh_arr < amp_min_anom_thresh))
if var_anom_minthresh_arr.shape[1] > 0:
    for var_anom_minthresh_lat, var_anom_minthresh_lon in zip(var_anom_minthresh_arr[0], var_anom_minthresh_arr[1]):
        var_anom_thresh_arr[var_anom_minthresh_lat, var_anom_minthresh_lon] = amp_min_anom_thresh

In [135]:
zg_dtrnd_norm_concat

<xarray.DataArray '500zg' (time: 3772, latitude: 46, longitude: 51)>
array([[[ -19.024953,  -14.202639, ...,   15.995887,   12.627666],
        [  -9.046545,   -3.793881, ...,   20.378644,   16.193107],
        ...,
        [-121.58304 , -117.623154, ..., -113.253671, -109.402385],
        [-124.210894, -121.396968, ..., -116.11093 , -113.487592]],

       [[ -74.006541,  -72.279716, ...,   67.6553  ,   67.604186],
        [ -69.200245,  -67.249532, ...,   74.467147,   74.286479],
        ...,
        [-104.427953,  -90.650472, ..., -146.018487, -142.672662],
        [-102.187412,  -88.509315, ..., -147.897308, -145.389288]],

       ...,

       [[ -52.060881,  -51.875366, ...,  141.285607,  143.290525],
        [ -73.9818  ,  -73.87225 , ...,  142.477941,  144.630154],
        ...,
        [ -45.666772,  -42.143968, ...,  -29.268138,  -27.856335],
        [ -44.200086,  -42.988199, ...,  -38.585354,  -36.758202]],

       [[ -25.381098,  -25.432213, ...,  101.905311,  106.16822 ],
  

In [134]:
var_anom_thresh_arr

<xarray.DataArray (latitude: 46, longitude: 51)>
array([[ 977.282884,  981.233441,  985.403726, ..., 1195.179679, 1199.424407,
        1203.566722],
       [ 978.697284,  982.650715,  986.739907, ..., 1178.556845, 1182.923702,
        1187.251519],
       [ 984.430355,  988.481792,  992.673386, ..., 1162.434417, 1166.925271,
        1171.445561],
       ...,
       [ 814.547155,  810.027419,  801.826074, ...,  650.888854,  642.684567,
         633.631427],
       [ 789.673955,  787.94226 ,  782.119589, ...,  628.374978,  619.947767,
         610.476846],
       [ 761.787765,  756.933808,  746.536363, ...,  609.086967,  598.833628,
         590.110009]])
Coordinates:
  * latitude   (latitude) float32 75.0 74.0 73.0 72.0 ... 33.0 32.0 31.0 30.0
  * longitude  (longitude) float32 -10.0 -9.0 -8.0 -7.0 ... 37.0 38.0 39.0 40.0

In [160]:
#create the boolean array showing the positive regions - can take a few seconds to run
zg_day_bool_thresh = [zg_day*9.80616 > var_anom_thresh_arr for zg_day in zg_dtrnd_norm_concat[:]] 

In [ ]:
zg_day_bool_thresh_concat.rename({'__xarray_dataarray_variable__': '500zg'})

In [161]:
#concatenate the lists of boolean DataArray objects - also takes a few seconds
zg_day_bool_thresh_concat = xarray.concat(zg_day_bool_thresh, "time")

In [162]:
zg_day_bool_thresh_concat#['time']

<xarray.DataArray (time: 3772, latitude: 46, longitude: 51)>
array([[[False, False, ..., False, False],
        [False, False, ..., False, False],
        ...,
        [False, False, ..., False, False],
        [False, False, ..., False, False]],

       [[False, False, ..., False, False],
        [False, False, ..., False, False],
        ...,
        [False, False, ..., False, False],
        [False, False, ..., False, False]],

       ...,

       [[False, False, ...,  True,  True],
        [False, False, ...,  True,  True],
        ...,
        [False, False, ..., False, False],
        [False, False, ..., False, False]],

       [[False, False, ..., False, False],
        [False, False, ..., False, False],
        ...,
        [False, False, ..., False, False],
        [False, False, ..., False, False]]])
Coordinates:
  * longitude  (longitude) float32 -10.0 -9.0 -8.0 -7.0 ... 37.0 38.0 39.0 40.0
  * latitude   (latitude) float32 75.0 74.0 73.0 72.0 ... 33.0 32.0 31.0 30.0
  * tim

In [164]:
xr.Dataset(zg_day_bool_thresh_concat.values)

xr.Dataset(data_vars=None, coords=None, attrs=None, compat=None)

TypeError: unhashable type: 'numpy.ndarray'

In [139]:
#define the attrs for the new xarray
zg_attrs = OrderedDict()
zg_attrs['units'] = 'm'
zg_attrs['long_name'] = 'Geopotential height'
zg_attrs['level'] = '500 hPa'
zg_attrs['dataset'] = 'era5'
zg_attrs['anom'] = 'subtract long term daily mean (see Pinheiro et al 2019)'
zg_attrs['detrended'] = '500 hPa vs surface temperature EurAR5 fldmean anom'
zg_attrs['norm'] = 'latitudinal coefficient (eq1 from Dole and Gordon 1983)'
zg_attrs['date_range'] = 'JJA 1979-2018'
zg_attrs['bool_thresh'] = 'True if value > 1.5*timstd or 100m (whichever is larger)'


zg_day_bool_thresh_concat = xr.Dataset()
zg_day_bool_thresh_concat.to_netcdf("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/tot/"
                                    "500zg_JJA_era5_1979-2019_daymean_EurAR5_1x1_LTDManom_dtrnd_wrt_tas_DG83_bool.nc")

# 3. Apply the area threshold
For both DG83 (500hPa geopotential height anomaly) and S04 (VPV anomaly).

Define the relevant ``var_bool`` file depending on what one is calculating, and then the functions

In [4]:
xr.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/tot/"
                                    "500zg_JJA_era5_1979-2019_daymean_EurAR5_1x1_LTDManom_dtrnd_wrt_tas_DG83_bool.nc")['__xarray_dataarray_variable__']

<xarray.DataArray '__xarray_dataarray_variable__' (time: 3772, latitude: 46, longitude: 51)>
[8849112 values with dtype=bool]
Coordinates:
  * longitude  (longitude) float32 -10.0 -9.0 -8.0 -7.0 ... 37.0 38.0 39.0 40.0
  * latitude   (latitude) float32 75.0 74.0 73.0 72.0 ... 33.0 32.0 31.0 30.0
  * time       (time) datetime64[ns] 1979-06-01T10:30:00 ... 2019-08-31T10:30:00

In [11]:
var_bool_orig.sum('time').sum('latitude').sum('longitude')

<xarray.DataArray '500zg' ()>
array(556593)

In [9]:
var_bool.sum('time').sum('latitude').sum('longitude')

<xarray.DataArray '__xarray_dataarray_variable__' ()>
array(572092)

In [10]:
#these files have already had the amplitude calculation applied
#var_bool = xarray.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/
#500zg_JJA_era5_1979-2018_0000_EurAR5_anom_dtrnd_wrt_tas_DG83_area_weighted_varbool.nc")['500zg']

var_bool_orig = xarray.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/"
                "500zg_JJA_era5_1979-2019_daymean_EurAR5_1x1_anom_dtrnd_wrt_tas_DG83_bool.nc")['500zg']
#for the redefined anomaly
var_bool = xarray.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/tot/"
                                    "500zg_JJA_era5_1979-2019_daymean_EurAR5_1x1_LTDManom_dtrnd_wrt_tas_DG83_bool.nc")['__xarray_dataarray_variable__']

In [6]:
time, lat, lon = var_bool['time'], var_bool['latitude'], var_bool['longitude']

In [13]:
def len_iter(items):
    return sum(1 for _ in items)

def consecutive_one(data):
    return max(len_iter(run) for val, run in groupby(data) if val)

In [14]:
#convert lat and lon coordinates of polygon to calculate polygon area
#from https://gis.stackexchange.com/questions/298929/converting-lat-lon-to-x-y-coordinates-with-pyproj
#alternative solution: https://stackoverflow.com/questions/4681737/how-to-calculate-the-area-of-a-polygon-on-the-earths-surface-using-python/4682656#4682656
P = pyproj.Proj(proj='utm', zone=31, ellps='WGS84', preserve_units=True)
G = pyproj.Geod(ellps='WGS84')

def LatLon_To_XY(Lat,Lon):
    return np.array(P(Lat,Lon))    

def XY_To_LatLon(x,y):
    return P(x,y,inverse=True)    

def distance(Lat1, Lon1, Lat2, Lon2):
    return G.inv(Lon1, Lat1, Lon2, Lat2)[2]

def area_of_polygon(x, y):
    """Calculates the area of an arbitrary polygon given its verticies"""
    area = 0.0
    for i in range(-1, len(x)-1):
        area += x[i] * (y[i+1] - y[i-1])
    return abs(area) / 2.0

In [15]:
#adapted from https://gis.stackexchange.com/questions/99917/converting-matplotlib-contour-objects-to-shapely-objects
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx+1

def area_test(var_bool, lat, lon, grid_coords, area_thresh, prop_blocked_thresh, min_num_grid_cells):
    """
    Calculate the polygon shapes, determine the area for the blocked polygons and if there is a
    blocked polygon identify whether or not it passes the area_thresh threshold
    if so, then day is blocked
    """
    blocked_day = 0
    prop_blocked = np.array(np.where(var_bool == 1)).shape[1]/(len(lat)*len(lon))
    if prop_blocked < prop_blocked_thresh:
        return blocked_day, [[0]]
    #test if there are two few blocked grid cells that we know there cannot be blocking
    CS = plt.contour(lon,lat,var_bool)
    plt.close()
    var_bool_blocked_idx = np.zeros((len(lat)*len(lon)))#use to identify blocked regions    
    for col in CS.collections:
        # Loop through all polygons that have the same intensity level
        for contour_path in col.get_paths():
            # Create the polygon for this intensity level
            # The first polygon in the path is the main one, the following ones are "holes"
            for ncp,cp in enumerate(contour_path.to_polygons()):
                
                try:
                    x = cp[:,0]
                    y = cp[:,1]
                except TypeError: #when the data is stored as a list of arrays
                    x = [arr[0] for arr in cp]
                    y = [arr[1] for arr in cp]
                new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])
                if ncp == 0:
                    poly = new_shape
                else:
                    # Remove the holes if there are any
                    print("Holes")
                    poly = poly.difference(new_shape)
                    # Can also be left out if you want to include all rings
                # do something with polygon
                # Extract the point values that define the perimeter of the polygon
                lon_shape, lat_shape = poly.exterior.coords.xy
                #make a path object for the polygon
                tup_verts = list(zip(lat_shape, lon_shape))
                p = Path(tup_verts) # make a polygon
                points = grid_coords
                grid = p.contains_points(points)
                mask = grid.reshape(len(lat),len(lon)) # now you have a mask with points inside a polygon            
                num_grid_cells_blocked_in_poly = sum(grid)
                if num_grid_cells_blocked_in_poly > min_num_grid_cells:
                    #blocked region potentially large enough to qualify as blocking
                    #calculate area of polygon by converting from lat,lon coordinates to x, y coordinates
                    x, y = LatLon_To_XY(lat_shape,lon_shape)
                    area = geometry.Polygon(list(zip(x, y))).area
                    #covnert from m^2 to km^2
                    area_km = area/1e6
                    if area_km > area_thresh:
                        #label day as blocked
                        blocked_day = 1
                        #relabel cells in blocked polygon for testing the overlap critereon
                        var_bool_blocked_idx += grid #add the array to a 1d level
    #return if the day is blocked and the flattened index values of the blocked tiles for the overlap test
    return blocked_day, np.where(var_bool_blocked_idx > 0)



In [16]:
#these variables can be modified
area_thresh = 1e6 # minimum blocked area in km^2
# 1e6 km^2 is a reasonable minimum area since this is approximately the squared radius of a typical anticyclone (pg 10 of Hoskins & James)
grid_res = 1 #grid resolution
#create a tuple of the lat/lon coordinates of the grid to test whether or not the grid cells exist within the point
lats, lons = np.asarray(np.meshgrid(lat,lon))[0,:,:].flatten(), np.asarray(np.meshgrid(lat,lon))[1,:,:].flatten()
grid_coords = list(zip(lats,lons))
#rough minimum for the number of grid cells needed for a blocking event
min_num_grid_cells=area_thresh*np.cos(np.radians(30))/(grid_res*grid_res*110*110) #0.25x0.25 grid x 110km (roughly 1/360 of Earth's circumference)
#minimum proportion of grid cells to block
prop_blocked_thresh = min_num_grid_cells/(len(lat)*len(lon))

In [17]:
var_bool

<xarray.DataArray '__xarray_dataarray_variable__' (time: 3772, latitude: 46, longitude: 51)>
[8849112 values with dtype=bool]
Coordinates:
  * longitude  (longitude) float32 -10.0 -9.0 -8.0 -7.0 ... 37.0 38.0 39.0 40.0
  * latitude   (latitude) float32 75.0 74.0 73.0 72.0 ... 33.0 32.0 31.0 30.0
  * time       (time) datetime64[ns] 1979-06-01T10:30:00 ... 2019-08-31T10:30:00

In [18]:
#note that for this dataset this cell takes a long time (several minutes) to run
lat = var_bool['latitude']
lon = var_bool['longitude']
time = var_bool['time']
blocked_day_arr = []
#save the block_arr_area and blocked_idx_vals files
#this gives information about the index values that are blocked on that day, which will be used when applying the overlap threshold
with open(f"/rds/general/project/carl_phd/live/carl/data/era5/day/zg/block_data/blocked_day/LTDM/blocked_day_arr_tot_LTDManom.txt", "a+") as blocked_day_arr_tot:
    for i in range(0,len(time)):
        if i%50 == 0:
            print(i)
        blocked_day, var_bool_blocked = area_test(var_bool[i,:,:], lat, lon, grid_coords, area_thresh, prop_blocked_thresh, min_num_grid_cells)
        #the block_data files saves the data for each day for the particular indices that are blocked
        np.savetxt(f"/rds/general/project/carl_phd/live/carl/data/era5/day/zg/block_data/LTDM/blocked_idx_vals_LTDManom_%04d.txt"%i, var_bool_blocked[0])        
        blocked_day_arr_tot.write("%s "%blocked_day)
        
blocked_day_arr_tot.close()

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750


In [19]:
sum(np.loadtxt("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/block_data/blocked_day/LTDM/blocked_day_arr_tot_LTDManom.txt"))

1809.0

In [20]:
blocked_day_arr_tot = np.loadtxt("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/block_data/blocked_day/blocked_day_arr_tot_newresdaymean.txt")
blocked_day_arr_tot = np.loadtxt("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/block_data/blocked_day/LTDM/blocked_day_arr_tot_LTDManom.txt")

# 4. Apply the persistence criterea
This is defined and then applied both before and after the overlap criterea

In [21]:
def search_sequence_numpy(arr,seq):
    """ Find sequence in an array using NumPy only.

    Parameters
    ----------    
    arr    : input 1D array
    seq    : input 1D array

    Output
    ------    
    Output : 1D Array of indices in the input array that satisfy the 
    matching of input sequence in the input array.
    In case of no match, an empty list is returned.
    """

    # Store sizes of input array and sequence
    Na, Nseq = len(arr), len(seq)

    # Range of sequence
    r_seq = np.arange(Nseq)

    # Create a 2D array of sliding indices across the entire length of input array.
    # Match up with the input sequence & get the matching starting indices.
    M = (arr[np.arange(Na-Nseq+1)[:,None] + r_seq] == seq).all(1)

    # Get the range of those indices as final output
    if M.any() >0:
        return np.where(np.convolve(M,np.ones((Nseq),dtype=int))>0)[0]
    else:
        return []         # No match found


In [22]:
#to apply the persstence critereon, need to remove all events that don't persist over a period of five days
#any sequence of 1,1,1,1,1 or more is fine, but 0,1,1,1,1,0 or less must be removed
#sequences to be removed: 0,1,1,1,1,0; 0,1,1,1,0; 0,1,1,0; 0,1,0
#note that Verdecchia et al (1996) add some criterea for blocking to be removed
def persistence_criterea(block_arr, persis_thresh, JJA_days):
    """
    Apply the persistence criterea to a list of days which are blocked or not
    by removing the sequences for days which are not blocked
    persis_thresh is the minimum number of blocked days for the threshold (at least 2)
    block_arr is an array of True/False days of blocking
    NB removes when blocking occurs briefly at the start or end of a season
    Assume that blocking at the end of August doesn't persist into early September
    and that there was no blocking late May. This would be a sensible approximation to apply
    since we are only here interested in JJA blocking events.
    """
    #pad the array to remove any erroneous blocking events that would coincide from the end of the season to the start of the next
    block_arr_pad = block_arr.copy()
    #to add elements to an array, add from reverse so that the index numbering doesn't mess up
    idx_arr = np.arange(JJA_days, block_arr.shape[0], JJA_days)[::-1]

    for i in idx_arr:
        block_arr_pad = np.concatenate((block_arr_pad[:i], [0], block_arr_pad[i:]))
        
    block_arr_persis=block_arr.copy()
    #blocking sequences to be removed
    seq_arr = [[0,1,0], [0,1,1,0], [0,1,1,1,0], [0,1,1,1,1,0], [0,1,1,1,1,1,0]]
    seq_to_remove_arr = seq_arr[:persis_thresh-1]
    

    block_arr_pad_persis=block_arr_pad.copy()
    for seq in seq_to_remove_arr:
        seq_idx=search_sequence_numpy(block_arr_pad_persis,seq)
        block_arr_pad_persis[seq_idx] = 0
    # then need to remove the padding from the array
    #indices to be removed - note the shift of the index ordering
    added_idx = np.arange(JJA_days, block_arr.shape[0], JJA_days+1)
    block_arr_persis = np.delete(block_arr_pad_persis, added_idx)
    return block_arr_persis

In [23]:
#load the data from DG83
block_idx_vals_file_list = sorted(glob.glob("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/block_data/*blocked_idx_vals*.txt"))
block_idx_vals_file_list = sorted(glob.glob("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/block_data/*newresdaymean*"))
block_idx_vals_file_list = sorted(glob.glob("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/block_data/LTDM/*blocked_idx_vals*.txt"))

In [25]:
#apply persistence criterea
#this seems to remove half of all blocking events and can take a few minutes to run
blocked_day_arr_persis = persistence_criterea(np.array(blocked_day_arr_tot), persis_thresh, JJA_days)

In [30]:
sum(blocked_day_arr_persis)/len(blocked_day_arr_persis)

0.3297985153764581

# 5. Apply the overlap criterea

In [28]:
#potentially helpful: https://stackoverflow.com/questions/55641425/check-if-two-contours-intersect
def overlap_contour_criterea(blocked_idx_vals, block_arr, persis_thresh, JJA_days, overlap_val):
    """
    To calculate the overlap criterea, need contour overlap over the five day period
    For each five day period, need contour overlap
    for the region to remain blocked over that five day period.
    """
    #block_arr = block_arr_orig.copy() #change back when not testing
    for i, blocked in enumerate(block_arr):
        # measure if the previous day was blocked
        if i == 0: 
            prev_blocked = 0
        else:
            prev_blocked = block_arr[i-1]
        if blocked == 1 and prev_blocked == 0: #if a new blocking event is starting
            #measure the length of the blocked event (in days) by finding the first zero element after the first zero element
            blocked_arr_samp = block_arr[i:]
            length_of_block = next((index for index,value in enumerate(blocked_arr_samp) if value != 1), 0)
            #need to sample periods of persis_thresh within this blocked period
            #and test whether or not the contours overlap, that is whether or not a certain percentage of the
            #indices of the blocked longitudes are identical
            for k in range(i,i+length_of_block-persis_thresh+1):#loop within the blocked period across the minumum persistence criterea
                #since the shorter periods have been removed, blocked_idx_arr shows the indices of all of the blocked zones on each day
                #within the minimum time period
                #blocked_idx_vals is a 1d list for the blocked indices
                #calculate the number of tiles blocked across the period
                blocked_across_period = calc_blocked_overlap_across_period(blocked_idx_vals[k:k+persis_thresh], overlap_val)
                #if the number of blocked cells is less than the number of overlapping cells
                #then there isn't a contour overlap across the five day period
                #so need to find where the discontinuity takes place within this time period
                if blocked_across_period < overlap_val:
                    while blocked_across_period < overlap_val:
                        for l in range(1, persis_thresh-1):
                        #subselect periods of blocked days to find at what point the deviation occurs
                        #and then eliminate previous day as a block
                            blocked_idx_vals_subsamp_list = array_subsampled(blocked_idx_vals[k:k+persis_thresh-1], l)
                            try:
                                for m, blocked_idx_vals_subsamp in enumerate(blocked_idx_vals_subsamp_list):
                                    block_arr[l+k-1+m] = 0
                                    #when there is only one value in blocked_idx_vals_subsamp
                                    #eliminate previous day as a block for each time when there isn't a block calculated across the period     
                                    blocked_across_period = calc_blocked_overlap_across_period(blocked_idx_vals_subsamp, overlap_val)
                            except TypeError: #exception when there is only one list
                                    block_arr[l+k-1] = 0
                                    blocked_across_period = calc_blocked_overlap_across_period(blocked_idx_vals_subsamp_list, overlap_val)
                        blocked_across_period = overlap_val # to get out of the while loop if needed (at this point no overlap has been identified)
    return block_arr

In [29]:
def calc_blocked_overlap_across_period(blocked_idx_vals, overlap_val):
    """
    Calculate the number of tiles blocked across period
    """
    blocked_idx_across_period = []
    for num_days, blocked_idx_day in enumerate(blocked_idx_vals):
        blocked_idx_across_period = np.append(blocked_idx_across_period, blocked_idx_day)
    #count the number of elements that occur num_days+1 times
    blocked_idx_unique, return_counts_arr = np.unique(blocked_idx_across_period, return_counts=True)
    blocked_across_period = len([j for i,j in enumerate(blocked_idx_unique) if return_counts_arr[i] > num_days])
    return blocked_across_period

def array_subsampled(arr, i):
    """
    Return consecutive samples of an array
    """
    arr_list = []
    
    #there are i numbers of array
    for j in range(i+1):
        #0:len(arr)-i, 1:len(arr)-i+1, ... , i-1:len(arr)-1, i:len(arr)
        arr_list = np.append(arr_list, arr[0+j:len(arr)-i+j])
    return arr_list

In [31]:
#calculate overlap - usually takes a few seconds
blocked_day_arr_persis_overlap = overlap_contour_criterea(blocked_idx_vals, blocked_day_arr_persis, persis_thresh, JJA_days, overlap_val)

In [32]:
#run the persistence threshold again to remove any blocking events that now no longer pass the persistence criterea
blocked_day_arr_persis_overlap_persis = persistence_criterea(np.array(blocked_day_arr_persis_overlap), persis_thresh, JJA_days)

In [33]:
sum(blocked_day_arr_persis_overlap_persis)/len(blocked_day_arr_persis_overlap_persis)

0.12963944856839874

In [34]:
np.savetxt("/rds/general/user/cmt3718/home/data/DG83_idx/DG83_idx_block_arr_era5_daymean_LTDM.txt"  , blocked_day_arr_persis_overlap_persis)

In [ ]:
TM90=np.loadtxt("")

In [49]:
sum(DG83_orig), sum(DG83_LTDM)

(534.0, 489.0)

In [ ]:
DG83_LTDM=np.loadtxt("/rds/general/user/cmt3718/home/data/DG83_idx/DG83_idx_block_arr_era5_daymean_LTDM.txt")
DG83_orig=np.loadtxt("/rds/general/user/cmt3718/home/data/DG83_idx/DG83_idx_block_arr_era5_daymean.txt")

In [47]:
(np.where(DG83_orig[:] < DG83_LTDM[:])[0]) #list of days classified in LTDM which aren't in the original dataset

array([  71,   93,   94,   95,   96,   97,   98,   99,  100,  121,  122,
        123,  124,  125,  135,  136,  137,  138,  139,  221,  222,  223,
        224,  225,  331,  521,  522,  523,  524,  525,  619,  620,  621,
        622,  623,  624,  625,  626,  627,  628,  629,  656,  657,  658,
        659,  660,  661,  663,  664,  665,  666,  667,  763,  764,  765,
        766,  767,  768, 1205, 1206, 1207, 1208, 1209, 1260, 1261, 1262,
       1263, 1264, 1265, 1266, 1267, 1268, 1412, 1413, 1414, 1415, 1416,
       1572, 1573, 1574, 1630, 1631, 1632, 1633, 1634, 1635, 1636, 1637,
       1638, 1649, 1650, 1651, 1652, 1653, 1654, 1655, 1694, 1695, 1701,
       1702, 1703, 1704, 1727, 1728, 1739, 1868, 1941, 1942, 1943, 1944,
       1945, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2120,
       2121, 2192, 2358, 2367, 2368, 2369, 2370, 2371, 2372, 2373, 2641,
       2668, 2669, 2670, 2671, 2672, 2673, 2674, 2720, 2822, 2823, 2824,
       2825, 2826, 2827, 2828, 2829, 2891, 2892, 28

In [40]:
np.where(DG83_orig[:] == DG83_LTDM[:])

(array([   0,    1,    2, ..., 3769, 3770, 3771]),)

In [43]:
scipy.stats.pearsonr(DG83_orig,DG83_LTDM)

(0.4885049010410959, 1.6785941972795775e-225)

NameError: name 'xr' is not defined